<a id=contents></a>

# Extraction and cleaning notebook
## Data retrieved from glassdoor.com

Data was extracted via Glassdoor's REST API (documentation here: https://www.glassdoor.com/developer/index.htm). 

[0. Data extraction via Selenium scraping](#api)

[1. Data Inspection](#insp)

[2. Cleaning numerical data](#numerical)

[3. Cleaning categorical data](#categ)

[4. Cleaning text data](#text)

In [56]:

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc

import os
import time
import requests as req
from dotenv import load_dotenv
load_dotenv()
import selenium as sl
from selenium.common.exceptions import ElementClickInterceptedException


import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("darkgrid")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.linear_model import LogisticRegression
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import string

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id=api ><a/> 

## 0. Data Extraction via Selenium scraping
    
[LINK to table of contents](#contents)

In [6]:
class APICaller:
    def __init__(self, base_url, token=None, ignore_token=True):
        self.token = os.getenv('TOKEN')
        if ignore_token==False:
            if len(self.token) == 0:
                raise ValueError('Missing API token!')
        self.base_url=base_url
        
    def retrieve_one(self,url_extension,location=None, date=None, date1=None):  
        if date1!=None:
            response = req.get(self.base_url+url_extension+f'{location}/StartDate={date}/EndDate={date1}/Json').json()
        elif (date!=None and date1==None):
            response = req.get(self.base_url+url_extension+f'{location}/Date={date}/Json').json()
        else:
            print(self.base_url+url_extension)
            response = req.get(self.base_url+url_extension).json()
        return response
    
    
    def retrieve_many(self,location_list, date_list, var, limit):
        data = []
        counter=0
        for location in location_list:
            for date in date_list:
                if counter==limit-1:
                    time.sleep(60)
                response = req.get(f'{self.url}/{key}/{location}/{date}/{var}').json()
                data.append(response)
                counter+=1
        data_df = pd.read_json(data)    
        return data_df

<a href='https://www.glassdoor.com/index.htm'>powered by <img src='https://www.glassdoor.com/static/img/api/glassdoor_logo_80.png' title='Job Search' /></a>

In [10]:
from selenium import webdriver


In [61]:
driver = webdriver.Chrome('/Users/ipreoteasa/Desktop/Io/chromedriver_2')


In [62]:
base_url = 'https://www.glassdoor.co.uk/'
# ext_job = 'Job/london-data-scientist-jobs-SRCH_IL.0,6_IC2671300_KO7,21.htm?'
ext_type = 'Job/'
ext_search_loc = 'london-'
ext_job_title = 'data-scientist-'
ext_end = 'jobs-SRCH_IL.0,6_IC2671300_KO7,21.htm?'

driver.get(base_url+ext_type+ext_search_loc+ext_job_title+ext_end)

In [63]:
# getting a job descr by xpath

jobdescr_element = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div/div[1]/div/div/div[4]/div/div/div/div')[0]

# _xpath('//*[@id="Comment_5561090"]/div/div[2]/div[1]/span[1]/a[2]')[0]


jobdescr = jobdescr_element.text

IndexError: list index out of range

In [22]:
jobdescr

'We are looking for Data Scientists who are interested in using data to draw insights that will result in policy changes or business process optimisation, benefiting the public. The applicant will be scoping projects with stakeholders, using data sets across Government Agencies, applying business acumen to tease out relevant impactful insights, and presenting insights in a clear, concise manner by using appropriate visualisations.\n\nHe/she should have some training and working experiences on data analytics, and should be comfortable with hands-on data manipulation, data modelling and data visualisation. He/she should also be comfortable with engaging stakeholders on sharpening their business problems.\n\nThe analytics work that we do are typically action oriented and cross-cutting across various domains such as social, economic and infrastructure sectors. Over time, he/she will gain exposure to various policy and ops domains and become more adept in bridging between business users and

Ok so far so good, accessing the correct data. Now to turn to pagination and getting other resuts.

In [29]:
driver.current_url


'https://www.glassdoor.co.uk/Job/london-data-scientist-jobs-SRCH_IL.0,6_IC2671300_KO7,21.htm?'

In [80]:
base_url = 'https://www.glassdoor.co.uk/'
ext_type = 'Job/'
ext_search_loc = 'london-'
ext_job_title = 'data-scientist-'
ext_end = 'jobs-SRCH_IL.0,6_IC2671300_KO7,21.htm?'

# driver.get(base_url+ext_type+ext_search_loc+ext_job_title+ext_end)

def job_POST_scraper(base_url = base_url, ext_search_type=ext_type,
               ext_search_location = ext_search_loc, 
               ext_job_title = ext_job_title, 
               ext_end = ext_end, 
               num_job_descr=10):
    
    
    options = webdriver.ChromeOptions()
#     options.add_argument('headless')

    driver = webdriver.Chrome('/Users/ipreoteasa/Desktop/Io/chromedriver_2',
                             options=options)
    driver.get(base_url+ext_type+ext_search_loc+ext_job_title+ext_end)


    job_links = []
       
    while len(job_hrefs_list)<num_job_descr:
        time.sleep(4)
        try: 
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)
        # need to add a loop and a way to paginate through the next few jobs
#         job_column = driver.find_element_by_class_name("jlGrid hover")
# #         print(job_column.text)
#         job_hrefs_list = set([job_square.get_attribute('href') for job_square in job_column])# access the href element within the pages
        #get the driver to get the URL you've saved
        
        elems = driver.find_elements_by_xpath("//*[@id='MainCol']/div[1]/ul")
        for elem in elems:
            print(elem.get_attribute("href"))
            job_links.append(elem.text)
        
        
        time.sleep(5)
#         job_links = job_hrefs_list.text
#         print(job_links)
    return job_links

In [81]:
job_link_list = job_scraper()

We are looking for Data Scientists who are interested in using data to draw insights that will result in policy changes or business process optimisation, benefiting the public. The applicant will be scoping projects with stakeholders, using data sets across Government Agencies, applying business acumen to tease out relevant impactful insights, and presenting insights in a clear, concise manner by using appropriate visualisations.

He/she should have some training and working experiences on data analytics, and should be comfortable with hands-on data manipulation, data modelling and data visualisation. He/she should also be comfortable with engaging stakeholders on sharpening their business problems.

The analytics work that we do are typically action oriented and cross-cutting across various domains such as social, economic and infrastructure sectors. Over time, he/she will gain exposure to various policy and ops domains and become more adept in bridging between business users and tech

In [ ]:
def scrape_individual_job_posts():
    """Function that takes in list of job post links and scrapes the descriptions off"""
    
    
    pass

descr_list = []    
    
for link in job_link_list:
    descr_list.append(scrape_individual_job_posts(link=link))
    time.sleep(1)

<a id=insp ><a/> 

## 1. Data Inspection
    
[LINK to table of contents](#contents)

<a id=numerical ><a/> 

## 2. Cleaning numerical data
    
[LINK to table of contents](#contents)

<a id=categ ><a/> 

## 3. Cleaning categorical data
   
[LINK to table of contents](#contents)

<a id=text ><a/> 

## 4. Cleaning text data
    
[LINK to table of contents](#contents)